# NPC Decision Cycle Testing

This notebook provides full observability into the NPC cognitive pipeline, showing all intermediate processing steps, token usage, and timing.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import json
import time
import asyncio
from typing import Any
from dataclasses import dataclass
import logging

# Use the clean client instead of the one with LlamaIndex dependencies
from mind.agent.agent import Agent, AgentLLMConfig
from mind.apis.clean_llm_client import Model

# Set up logging to see all intermediate steps
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Token Counter

Simple token counting utility to measure usage at each step.

In [ ]:
import tiktoken

@dataclass
class TokenUsage:
    """Track token usage for a single LLM call"""
    step: str
    input_tokens: int
    output_tokens: int
    total_tokens: int
    duration_ms: int
    
class TokenTracker:
    """Track token usage across the pipeline"""
    def __init__(self):
        self.encoder = tiktoken.encoding_for_model("gpt-4")  # Close enough for Claude
        self.usage_history: list[TokenUsage] = []
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in a string"""
        return len(self.encoder.encode(text))
    
    def track_call(self, step: str, input_text: str, output_text: str, duration_ms: int):
        """Track a single LLM call"""
        usage = TokenUsage(
            step=step,
            input_tokens=self.count_tokens(input_text),
            output_tokens=self.count_tokens(output_text),
            total_tokens=self.count_tokens(input_text) + self.count_tokens(output_text),
            duration_ms=duration_ms
        )
        self.usage_history.append(usage)
        return usage
    
    def summary(self) -> dict[str, Any]:
        """Get summary of all token usage"""
        total_input = sum(u.input_tokens for u in self.usage_history)
        total_output = sum(u.output_tokens for u in self.usage_history)
        total_time = sum(u.duration_ms for u in self.usage_history)
        
        return {
            "total_input_tokens": total_input,
            "total_output_tokens": total_output,
            "total_tokens": total_input + total_output,
            "total_duration_ms": total_time,
            "steps": [{
                "step": u.step,
                "tokens": u.total_tokens,
                "duration_ms": u.duration_ms
            } for u in self.usage_history]
        }

## Observable Agent Wrapper

Wrap the Agent class to capture and display all intermediate processing.

In [ ]:
class ObservableAgent(Agent):
    """Agent wrapper that logs all intermediate steps for observation"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tracker = TokenTracker()
        self.intermediate_results = {}
    
    def _log_step(self, step_name: str, result: Any, input_text: str = "", duration_ms: int = 0):
        """Log and display intermediate step results"""
        print(f"\n{'='*60}")
        print(f"STEP: {step_name}")
        print(f"{'='*60}")
        
        if isinstance(result, dict):
            print(json.dumps(result, indent=2))
        elif isinstance(result, list):
            for i, item in enumerate(result):
                print(f"  [{i}]: {item}")
        else:
            print(result)
        
        # Track tokens if we have text
        if input_text and result:
            output_text = json.dumps(result) if isinstance(result, (dict, list)) else str(result)
            usage = self.tracker.track_call(step_name, input_text, output_text, duration_ms)
            print(f"\nTokens: {usage.total_tokens} (in: {usage.input_tokens}, out: {usage.output_tokens})")
            print(f"Time: {usage.duration_ms}ms")
        
        # Store for later analysis
        self.intermediate_results[step_name] = result
    
    def update_working_memory(self) -> None:
        """Override to capture intermediate results"""
        print("\n" + "#"*70)
        print("# UPDATING WORKING MEMORY")
        print("#"*70)
        
        # Current state
        print(f"\nCurrent Working Memory:\n{self.working_memory}\n")
        print(f"Current Observation:\n{self.current_observation}\n")
        
        # Step 1: Generate queries
        start_time = time.time()
        query_response = self.query_generator.generate(
            working_memory=self.working_memory,
            observation=self.current_observation,
        )
        duration = int((time.time() - start_time) * 1000)
        self._log_step("1. Memory Queries Generated", query_response, 
                      f"WM: {self.working_memory}\nObs: {self.current_observation}", duration)
        
        # Step 2: Retrieve memories
        retrieved_memories = []
        for query in query_response.get("queries", []):
            query_memories = self.long_term_memory.retrieve(query)
            if query_memories:
                retrieved_memories.append(f"Query: '{query}' -> Memory: '{query_memories[0]}'")
        self._log_step("2. Memories Retrieved", retrieved_memories)
        
        # Step 3: Generate memory report
        start_time = time.time()
        memory_report_response = self.memory_report_generator.generate(
            working_memory=self.working_memory,
            observation=self.current_observation,
            retrieved_memories=[m.split(" -> Memory: '")[1].rstrip("'") for m in retrieved_memories] if retrieved_memories else [],
        )
        duration = int((time.time() - start_time) * 1000)
        self._log_step("3. Memory Report Generated", memory_report_response,
                      f"WM: {self.working_memory}\nMemories: {retrieved_memories}", duration)
        
        # Step 4: Update working memory
        start_time = time.time()
        working_memory_response = self.working_memory_generator.generate(
            working_memory=self.working_memory,
            memory_report=memory_report_response["memory_report"],
        )
        duration = int((time.time() - start_time) * 1000)
        self._log_step("4. Working Memory Updated", working_memory_response,
                      f"Report: {memory_report_response['memory_report']}", duration)
        
        if working_memory_response["updated_working_memory"]:
            self.working_memory = working_memory_response["updated_working_memory"]
            print(f"\nNew Working Memory:\n{self.working_memory}")
    
    def update_long_term_memory(self) -> None:
        """Override to capture intermediate results"""
        print("\n" + "#"*70)
        print("# UPDATING LONG-TERM MEMORY")
        print("#"*70)
        
        start_time = time.time()
        memory_update_response = self.long_term_memory_generator.generate(
            working_memory=self.working_memory,
            observation=self.current_observation,
        )
        duration = int((time.time() - start_time) * 1000)
        self._log_step("5. Long-term Memories Generated", memory_update_response,
                      f"WM: {self.working_memory}\nObs: {self.current_observation}", duration)
        
        for memory in memory_update_response["memories"]:
            self.long_term_memory.add_memories([memory])
            print(f"  Added to LTM: {memory}")
    
    def choose_action(self) -> int:
        """Override to capture intermediate results"""
        print("\n" + "#"*70)
        print("# CHOOSING ACTION")
        print("#"*70)
        
        # Format available actions
        actions_str = "\n".join([
            f"- Action {idx}: {desc}"
            for idx, desc in self.current_actions.items()
        ])
        print(f"\nAvailable Actions:\n{actions_str}")
        
        # Get action decision
        start_time = time.time()
        next_action = self.action_decision_generator.generate(
            working_memory=self.working_memory,
            available_actions=actions_str,
            personality_traits=", ".join(self.personality_traits)
        )
        duration = int((time.time() - start_time) * 1000)
        self._log_step("6. Action Decision Made", next_action,
                      f"WM: {self.working_memory}\nActions: {actions_str}", duration)
        
        # Extract chosen action
        try:
            action_decision = json.loads(next_action["action_decision"])
            chosen_index = int(action_decision["action_index"])
            print(f"\nChosen Action: {chosen_index} - {self.current_actions[chosen_index]}")
            return chosen_index
        except (KeyError, ValueError, json.JSONDecodeError) as e:
            print(f"\nError parsing action: {e}")
            return list(self.current_actions.keys())[0]

## Test Scenario Setup

Create a realistic test scenario with preset agent state.

In [ ]:
# Configure LLMs
llm_config = AgentLLMConfig(
    small_llm=Model.GEMINI_FLASH,
    large_llm=Model.CLAUDE_SONNET,
)

# Initial agent state
initial_working_memory = """I am a blacksmith in the village. I've been working on a sword commission for the past two days. 
This morning I feel well-rested and ready to continue my work. My apprentice should arrive soon to help.
I need to finish the blade today and start on the handle tomorrow."""

initial_memories = [
    "Yesterday I shaped the blade and got it to the right length",
    "The customer wants a ceremonial sword with intricate engravings",
    "My apprentice has been learning quickly and can now help with basic tasks",
    "The forge needs more coal soon, running low on supplies",
    "Last week I completed a set of horseshoes for the stable",
    "The village festival is coming up in three days",
    "I promised my wife I would come home early today"
]

personality_traits = ["hardworking", "perfectionist", "patient teacher"]

# Test observation and actions
test_observation = """You are in your workshop at the forge. The morning sun streams through the window. 
Your apprentice just arrived and greets you cheerfully. The half-finished sword blade lies on your workbench. 
The forge fire has died down overnight and needs to be restarted. You hear the village bell tower chime 8 o'clock."""

test_actions = {
    0: "Greet your apprentice and ask them to restart the forge fire",
    1: "Immediately begin working on the sword blade yourself", 
    2: "Take a moment to inspect yesterday's work on the blade",
    3: "Go to the storage room to check coal supplies",
    4: "Start teaching your apprentice a new technique"
}

## Run Decision Cycle

Execute a single decision cycle with full observability.

In [ ]:
# Create observable agent
agent = ObservableAgent(
    llm_config=llm_config,
    initial_working_memory=initial_working_memory,
    initial_long_term_memories=initial_memories,
    personality_traits=personality_traits
)

print("="*70)
print("STARTING DECISION CYCLE")
print("="*70)
print(f"\nInitial State:")
print(f"  Working Memory: {len(initial_working_memory)} chars")
print(f"  Long-term Memories: {len(initial_memories)} memories")
print(f"  Personality: {personality_traits}")
print(f"\nObservation: {test_observation}")
print(f"\nAvailable Actions: {len(test_actions)} options")

# Run the decision cycle
start_time = time.time()
chosen_action = agent.process_observation(test_observation, test_actions)
total_time = time.time() - start_time

print("\n" + "="*70)
print("DECISION CYCLE COMPLETE")
print("="*70)
print(f"\nFinal Action Chosen: {chosen_action} - {test_actions[chosen_action]}")
print(f"Total Time: {total_time:.2f} seconds")

## Summary and Analysis

Display token usage summary and identify optimization opportunities.

In [ ]:
# Get token usage summary
summary = agent.tracker.summary()

print("="*70)
print("TOKEN USAGE SUMMARY")
print("="*70)
print(f"\nTotal Tokens: {summary['total_tokens']}")
print(f"  Input: {summary['total_input_tokens']}")
print(f"  Output: {summary['total_output_tokens']}")
print(f"\nTotal Time: {summary['total_duration_ms']}ms")

print("\nBreakdown by Step:")
for step in summary['steps']:
    print(f"  {step['step']:<40} {step['tokens']:>6} tokens  {step['duration_ms']:>6}ms")

# Identify redundancies
print("\n" + "="*70)
print("OPTIMIZATION OPPORTUNITIES")
print("="*70)

# Check for redundancy between memory report and working memory update
if 'memory_report' in agent.intermediate_results.get('3. Memory Report Generated', {}):
    memory_report = agent.intermediate_results['3. Memory Report Generated']['memory_report']
    working_memory_update = agent.intermediate_results.get('4. Working Memory Updated', {}).get('updated_working_memory', '')
    
    print("\n1. Memory Report vs Working Memory Update:")
    print(f"   Memory Report Length: {len(memory_report)} chars")
    print(f"   Working Memory Length: {len(working_memory_update)} chars")
    print(f"   Observation: These two steps are doing similar consolidation work")

# Check memory retrieval efficiency
retrieved = agent.intermediate_results.get('2. Memories Retrieved', [])
print(f"\n2. Memory Retrieval Efficiency:")
print(f"   Queries Generated: {len(agent.intermediate_results.get('1. Memory Queries Generated', {}).get('queries', []))}")
print(f"   Memories Retrieved: {len(retrieved)}")
print(f"   Observation: Currently only using first memory per query")

# Check if long-term memory formation is always necessary
ltm_formed = agent.intermediate_results.get('5. Long-term Memories Generated', {}).get('memories', [])
print(f"\n3. Long-term Memory Formation:")
print(f"   Memories Formed: {len(ltm_formed)}")
print(f"   Observation: Every observation creates memories - should be selective")

print("\n" + "="*70)